# California, 2015–2023

## Overview
- Pulls historical weather data from **Visual Crossing API** for traffic accidents in California.
- Maps weather conditions to a **custom risk score** (0–3) to assess accident severity correlation.
- Uses **800 random accident samples** from the FARS dataset for efficient analysis.


In [2]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import urllib.request
import urllib.parse
import json

In [3]:
# API settings
API_KEY = "T7KA7J54DPZKSY4E53M5MWQ9R"
BASE_URL = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/"

### `get_weather_vr(lat, lon, date_str)` Function

This function fetches weather for one location and date using the API.

If it works, it pulls temp, rain, wind, visibility, and conditions, if not, it just skips quietly.

In [4]:
def get_weather_vr(lat, lon, date_str):
    """Dohvati historijsko vrijeme za jednu točku i datum."""
    url = (
        f"{BASE_URL}{urllib.parse.quote(f'{lat},{lon}')}/{date_str}"
        f"?key={API_KEY}"
        f"&unitGroup=metric"
        f"&include=days"
        f"&elements=datetime,temp,precip,windspeed,visibility,conditions"
        f"&contentType=json"
    )
    try:
        with urllib.request.urlopen(url, timeout=15) as response:
            if response.status == 200:
                data = json.loads(response.read().decode('utf-8'))
                if "days" in data and len(data["days"]) > 0:
                    day = data["days"][0]
                    return {
                        "temp_c": day.get("temp"),
                        "precip_mm": day.get("precip"),
                        "windspeed_kmh": day.get("windspeed"),
                        "visibility_km": day.get("visibility"),
                        "weather_condition": day.get("conditions")
                    }
    except Exception as e:
        # print(f"Greška za {lat},{lon},{date_str}: {e}")
        pass
    return None

### `condition_to_risk(cond)` Function

- Converts weather condition text into a numerical risk score (0–3).
  - `0`: Clear/cloudy (no risk)
  - `1`: Light precipitation (low risk)
  - `2`: Rain/snow/ice/fog (moderate risk)
  - `3`: Severe weather (high risk)

In [6]:
def condition_to_risk(cond):
    if pd.isna(cond) or cond is None:
        return 0
    cond = cond.lower()
    if any(kw in cond for kw in ["clear", "partly cloudy", "partially cloudy", "cloudy", "overcast"]):
        return 0
    elif any(kw in cond for kw in ["mist", "light drizzle", "light rain", "drizzle"]):
        return 1
    elif any(kw in cond for kw in ["rain", "snow", "ice", "freezing", "fog"]):
        return 2
    elif any(kw in cond for kw in ["heavy rain", "thunderstorm", "hail", "snowstorm"]):
        return 3
    else:
        return 0

# Loading California Data

In [5]:
project_root = Path(os.getcwd()).parents[0]
data_folder = project_root / 'data' / 'data_accidents'

all_ca = []
for year in range(2015, 2024):
    acc_file = data_folder / str(year) / "accident.csv"
    if not acc_file.exists():
        continue
    try:
        df = pd.read_csv(acc_file, low_memory=False, encoding="utf-8")
    except:
        df = pd.read_csv(acc_file, low_memory=False, encoding="latin1")
    if "STATE" in df.columns:
        df = df[df["STATE"] == 6]  # CA = 6
        df = df[["ST_CASE", "LATITUDE", "LONGITUD", "YEAR", "MONTH", "DAY"]]
        all_ca.append(df)

full_ca = pd.concat(all_ca, ignore_index=True)
print(f"Total number of accidents in CA: {len(full_ca):,}")

Total number of accidents in CA: 32,958


In [7]:
# Take 800 random sample accidents
sampled_ca = full_ca.sample(n=800, random_state=42).copy()

In [11]:
sampled_ca = sampled_ca[
    (sampled_ca["LATITUDE"].notna()) &
    (sampled_ca["LONGITUD"].notna()) &
    (sampled_ca["MONTH"].between(1, 12)) &
    (sampled_ca["DAY"].between(1, 31))
]
sampled_ca["date"] = pd.to_datetime(sampled_ca[["YEAR", "MONTH", "DAY"]], errors="coerce")
sampled_ca = sampled_ca[sampled_ca["date"].notna()]
sampled_ca["date_str"] = sampled_ca["date"].dt.strftime("%Y-%m-%d")

print(sampled_ca.columns)
print(sampled_ca.head())
print(len(sampled_ca))

Index(['ST_CASE', 'LATITUDE', 'LONGITUD', 'YEAR', 'MONTH', 'DAY', 'date',
       'date_str'],
      dtype='object')
       ST_CASE   LATITUDE    LONGITUD  YEAR  MONTH  DAY       date    date_str
21101    60294  34.641319 -115.674389  2021      5    7 2021-05-07  2021-05-07
24692    63923  37.353036 -121.944994  2021      8   31 2021-08-31  2021-08-31
13637    63452  33.574547 -117.690692  2018     11   23 2018-11-23  2018-11-23
1337     61341  34.249911 -118.322556  2015      7   30 2015-07-30  2015-07-30
31435    62245  37.236219 -121.919519  2023      1    8 2023-01-08  2023-01-08
800


In [12]:
weather_data = []
for i, row in tqdm(sampled_ca.iterrows(), total=len(sampled_ca), desc="Fetching weather data"):
    w = get_weather_vr(row["LATITUDE"], row["LONGITUD"], row["date_str"])
    if w:
        weather_data.append({
            "index": i,
            "temp_c": w["temp_c"],
            "precip_mm": w["precip_mm"],
            "windspeed_kmh": w["windspeed_kmh"],
            "visibility_km": w["visibility_km"],
            "weather_condition": w["weather_condition"]
        })

# Pretvori u DataFrame i spoji
weather_df = pd.DataFrame(weather_data).set_index("index")
final_df = sampled_ca.join(weather_df)

Fetching weather data: 100%|██████████| 800/800 [08:58<00:00,  1.49it/s]


In [13]:
# Join with final_df
weather_df = pd.DataFrame(weather_data).set_index("index")
final_df = sampled_ca.join(weather_df)

In [14]:
# Adding risk score
final_df["weather_risk_score"] = final_df["weather_condition"].apply(condition_to_risk)
print(final_df.head())

       ST_CASE   LATITUDE    LONGITUD  YEAR  MONTH  DAY       date  \
21101    60294  34.641319 -115.674389  2021      5    7 2021-05-07   
24692    63923  37.353036 -121.944994  2021      8   31 2021-08-31   
13637    63452  33.574547 -117.690692  2018     11   23 2018-11-23   
1337     61341  34.249911 -118.322556  2015      7   30 2015-07-30   
31435    62245  37.236219 -121.919519  2023      1    8 2023-01-08   

         date_str  temp_c  precip_mm  windspeed_kmh  visibility_km  \
21101  2021-05-07    25.8      0.000           31.7           16.0   
24692  2021-08-31    20.0      0.000           22.3           15.0   
13637  2018-11-23    15.7      0.000           12.4           15.8   
1337   2015-07-30    25.6      0.000           20.1           16.0   
31435  2023-01-08    13.4      5.186           43.4           14.9   

            weather_condition  weather_risk_score  
21101                   Clear                   0  
24692                   Clear                   0  
13

In [15]:
# Saving the data
output = "fars_ca_2015_2023_800sample_visualcrossing.csv"
final_df.to_csv(output, index=False)

print(f"\nSaved: {output}")
print(f"Succesfully fetched data for: {len(weather_df)} / {len(sampled_ca)} accidents")
print("\nRisk Distribution:")
print(final_df["weather_risk_score"].value_counts().sort_index())
print("\nExample of a weather condition:")
print(final_df["weather_condition"].value_counts().head())


Saved: fars_ca_2015_2023_800sample_visualcrossing.csv
Succesfully fetched data for: 799 / 800 accidents

Risk Distribution:
weather_risk_score
0    791
2      9
Name: count, dtype: int64

Example of a weather condition:
weather_condition
Clear                     430
Partially cloudy          243
Rain, Partially cloudy     92
Rain, Overcast             20
Rain                        8
Name: count, dtype: int64


Oout of 800 accidents, 799 had weather data.
791 occurred in clear or partly cloudy conditions — only 9 in rain or fog.
This suggests most fatal crashes happen in good weather, likely because more driving happens then.

In [16]:
df = pd.read_csv("fars_ca_2015_2023_800sample_visualcrossing.csv")
df["is_adverse"] = df["weather_risk_score"] >= 2

monthly = df.groupby("MONTH")["is_adverse"].mean()
print(monthly.sort_values(ascending=False))

MONTH
12    0.034483
3     0.017857
1     0.017544
11    0.014925
10    0.013699
7     0.013158
9     0.013158
2     0.000000
4     0.000000
5     0.000000
6     0.000000
8     0.000000
Name: is_adverse, dtype: float64
